### First read in the data

In [1]:
import pandas as pd

file_path = "D:/Laptop Data/Louis Box 2664/Documents/gym data project/weight change/01-07-19 weight bulk.csv"
wc = pd.read_csv(file_path)

### Lets take look at what we've got

In [2]:
wc.head(10)

,Date,Weight (lbs),Unnamed: 2,Unnamed: 3
0,01/07/2019,180.06,NaN,NaN
1,02/07/2019,181,NaN,NaN
2,03/07/2019,181,NaN,NaN
3,04/07/2019,180,NaN,NaN
4,05/07/2019,179.6,NaN,NaN
5,06/07/2019,181.8,NaN,NaN
6,07/07/2019,181,NaN,NaN
7,NaN,NaN,NaN,NaN
8,week 1 average,NaN,180.64,0.58
9,NaN,NaN,NaN,NaN


In [3]:
wc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           80 non-null     object 
 1   Weight (lbs)   70 non-null     object 
 2   Unnamed: 2     11 non-null     float64
 3   Unnamed: 3     12 non-null     object 
dtypes: float64(1), object(3)
memory usage: 3.2+ KB


### We've got a few things to work on. There a two columns which we dont need, they seem to be used to perform some calculations on average,difference etc.

So lets drop the third column onwards. Remember to set inplace=True so that the dataframe is actually changed.

In [4]:
wc.drop(columns = wc.columns[2:], inplace =True)

There are rows that act as visual seperators before and after each week. Lets remove them using Pandas' pd.dropna. Setting how = "all" drops rows where every value is Na.

In [5]:
wc.dropna(how="all",inplace=True)

### Looks good

In [6]:
wc.head(10)

,Date,Weight (lbs)
0,01/07/2019,180.06
1,02/07/2019,181
2,03/07/2019,181
3,04/07/2019,180
4,05/07/2019,179.6
5,06/07/2019,181.8
6,07/07/2019,181
8,week 1 average,NaN
10,08/07/2019,182.6
11,09/07/2019,182.7


Before we start the next step, we need to make sure we reset our index, since because we removed numerous rows we no longer have a continuous index. As you can see above we are missing row 7 & 9.

In [7]:
wc.reset_index(drop=True,inplace=True)

### Next we want to get rid of the "week x average" rows. Since they come at the end of every 7 days, we can drop every 7th row.

In [8]:
wc.drop(labels = wc.iloc[7::8].index.to_list(), inplace=True) # STARTING FROM INDEX 7 AND UP TO EVERY 8TH ITEM

In [9]:
wc.reset_index(drop=True,inplace=True)

### After a quick name change for the columns. Lets convert the weight column to numeric values using pd.to_numeric, as this will allow us to perform calculations on them.

In [11]:
wc.columns = ["Date","Weight"]

In [16]:
wc["Weight"] = wc["Weight"].apply(pd.to_numeric)

ValueError: Unable to parse string "183.8?" at position 0

### Python has thrown us a ValueError. Looks like not every value contains purely digits. lets have a look at what and how many items are going to cause an issue.

In [12]:
l=[]
for n in wc["Weight"]:
    try:
        pd.to_numeric(n)
    except ValueError:
        l.append(n)  
l

['183.8?', '184.6!', '188.6?', '189.6??', '191;']

### This shouldn't be hard to fix

In [8]:
#wc["Date"] = wc["Date"].apply(pd.to_datetime)

In [13]:
def fix_weight(n):
    if type(n) == str:
        fix = ""
        for v in n:
            if v.isdigit() or v==".":
                fix += v
        return fix
    else:
        return n

In [14]:
wc["Weight"] = wc["Weight"].apply(fix_weight)

### That seemed to work!

###  Now lets add a new column that indicates whether I was trying to gain, lose, maintain weight etc
To do this is simple, we can look at the file name and fill in value based on that.

In [19]:
def BulkCutFill(file):
    if "bulk" in file.lower():
        wc["Goal"] = "Bulk"
    elif "mini" in file.lower():
        wc["Goal"] = "Mini-cut"
    elif "cut" in file.lower():
        wc["Goal"] = "Cut"
    elif "maintain" in file.lower():
        wc["Goal"] = "Maintain"

In [22]:
BulkCutFill(file_path)

### The date format that we have currently isnt the best. Ideally we would want it in the format dd/mm/yyyy.
This function converts the string to datetime based on the format "%d/%m/%Y". Then back to a string with the new format "%Y-%m-%d".

In [24]:
import datetime
def convert_date_format(n):
    return datetime.datetime.strptime(n, "%d/%m/%Y").strftime("%Y-%m-%d") # "Y" MEANS YYYY AND "y"MEANS YY.

In [25]:
wc["Date"] = wc["Date"].apply(convert_date_format)

### Now the dataframe is looking much better.

In [26]:
wc.head(10)

,Date,Weight,Goal
0,2019-07-01,180.06,Bulk
1,2019-07-02,181,Bulk
2,2019-07-03,181,Bulk
3,2019-07-04,180,Bulk
4,2019-07-05,179.6,Bulk
5,2019-07-06,181.8,Bulk
6,2019-07-07,181,Bulk
7,2019-07-08,182.6,Bulk
8,2019-07-09,182.7,Bulk
9,2019-07-10,182.8,Bulk


### The last thing to do is loop through all the files and concatenate them into a single dataframe.

### Now I'll spare you the trouble of watching me attempt to debug the process of looping through the files. But one of the issues that I ran into was that the function os.scadir() from the OS library, would read in the files in a random order. Which is an issue for us as since we are concatenating everything into a single file, the order matters.

### There isn't an easy way of sorting dates in string format, since most sorting algorithms do so lexicographically and therefore will interpret the date "10/6/19" to be before "2/6/19" as "1" is less than "2". 

In [ ]:
def order_dates(folder_path): # TAKES IN THE FOLDER PATH OF THE FILES TO SORT
    
    split_date = [ n.split(" ",1)[0] for n in os.listdir(folder_path)] # CREATE A LIST OF THE DATES IN THE FILE NAME
                                                                       # BY SPLITTING ONCE ON A SPACE

    date_order = [datetime.datetime.strptime(i, "%d-%m-%y") for i in split_date] # CONVERT DATE STRINGS TO DATETIME
    date_order.sort() # SORT THEM

    string_order = [datetime.datetime.strftime(i, "%d-%m-%y") for i in date_order] # CONVERT BACK TO STRING

    full_string = [str(n) for n in os.listdir(folder_path)] # LIST OF FILE NAMES

    file_order = [i for n in string_order for i in full_string if n in i] # REORDER LIST OF FILE NAMES
    return file_order

### And putting it all together with a few tweaks we get this.

In [4]:
import pandas as pd
import numpy as np
import os
import datetime

def order_dates(folder_path):
    split_date = [ n.split(" ",1)[0] for n in os.listdir(folder_path)]

    date_order = [datetime.datetime.strptime(i, "%d-%m-%y") for i in split_date]
    date_order.sort()

    string_order = [datetime.datetime.strftime(i, "%d-%m-%y") for i in date_order]

    full_string = [str(n) for n in os.listdir(folder_path)]

    file_order = [i for n in string_order for i in full_string if n in i]
    return file_order

def fix_weight(n):
    if type(n) == str:
        fix = ""
        for v in n:
            if v.isdigit() or v==".":
                fix += v
        return fix
    else:
        return n

def BulkCutFill(file):
    if "bulk" in file.lower():
        wc["Goal"] = "Bulk"
    elif "mini" in file.lower():
        wc["Goal"] = "Mini-cut"
    elif "cut" in file.lower():
        wc["Goal"] = "Cut"
    elif "maintain" in file.lower():
        wc["Goal"] = "Maintain"
        
def convert_date_format(n):
    return datetime.datetime.strptime(n, "%d/%m/%Y").strftime("%Y-%m-%d")

folder_path = "D:/Laptop Data/Louis Box 2664/Documents/gym data project/weight change/"

col = ["Date","Weight","Goal"]

concat_df = pd.DataFrame(columns = col)

files = [folder_path + n for n in order_dates(folder_path)]

def full_cleaning():
    for f in files:

        wc = pd.read_csv(f)

        wc.drop(columns = wc.columns[2:], inplace =True)
        wc.dropna(how="all",inplace=True)
        wc.reset_index(drop=True,inplace=True)

        wc.drop(labels = wc.iloc[7::8, :].index.to_list(), inplace=True)
        wc.reset_index(drop=True,inplace=True)

        wc.columns = ["Date","Weight"]

        wc["Weight"] = wc["Weight"].apply(fix_weight)
        wc["Weight"] = wc["Weight"].apply(pd.to_numeric)    
        wc["Date"] = wc["Date"].apply(convert_date_format
                                     )
        BulkCutFill(f)

        concat_df = pd.concat([concat_df,wc],ignore_index=True)    
        concat_df["Weight"].fillna(method="ffill",inplace=True) 

        #np.savetxt("D:/Laptop Data/Louis Box 2664/Documents/gym data project/cleaned weight log/clean weight log.csv", 
         #       concat_df, fmt="%s", header="Date,Weight,Goal", comments="", delimiter=',')

if __name__ == "__main__":
    full_cleaning()